In [1]:
import tensorflow as tf

In [2]:
(xtrain, ytrain), (xtest, ytest) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
def net():
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters = 6, kernel_size=5, activation='sigmoid', padding='same'),
        tf.keras.layers.AvgPool2D(pool_size=2, strides=2),
        tf.keras.layers.Conv2D(filters = 16, kernel_size=5, activation='sigmoid'),
        tf.keras.layers.AvgPool2D(pool_size=2, strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation = 'sigmoid'),
        tf.keras.layers.Dense(84, activation = 'sigmoid'),
        tf.keras.layers.Dense(10)])

In [10]:
X = tf.random.uniform((1, 28, 28, 1))
for layer in net().layers:
    X = layer(X)
    print(layer.__class__.__name__, 'output_shape: \t', X.shape)

Conv2D output_shape: 	 (1, 28, 28, 6)
AveragePooling2D output_shape: 	 (1, 14, 14, 6)
Conv2D output_shape: 	 (1, 10, 10, 16)
AveragePooling2D output_shape: 	 (1, 5, 5, 16)
Flatten output_shape: 	 (1, 400)
Dense output_shape: 	 (1, 120)
Dense output_shape: 	 (1, 84)
Dense output_shape: 	 (1, 10)
